# HIV Model

##### Olivia Seitelman

In [3]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [6]:
state = State(R=1, L=1, E=1, V=1)
system = System(gamma=1.36, mu=1.36e-3, tau=0.2, beta=0.00027, p=0.1, alpha=3.6e-2, sigma=2, delta=0.33, pi=100, t_0=0, \
               t_end = 100, t_prev=0, t_current=0)

,values
gamma,1.36000
mu,0.00136
tau,0.20000
beta,0.00027
p,0.10000
alpha,0.03600
sigma,2.00000
delta,0.33000
pi,100.00000
t_0,0.00000


In [8]:
def update_func(state, t, system):
    """Update the HIV model.
    
    state: State (R, L, E, V)
    t: time
    system: System object
    
    returns: State (hiv)
    """
    R, L, E, V = state
    unpack(system)

    dR_dt = gamma*tau - mu*R - beta*R*V
    dL_dt = p*beta*R*V - mu*L - alpha*L
    dE_dt = (1-p)*beta*R*V + alpha*L -delta*E
    dV_dt = pi*E - sigma*V
    
    t_prev = t_current
    t_current = t
    dt = t_current - t_prev
    
    R = R + dR_dt*dt
    L = L + dL_dt*dt
    E = E + dE_dt*dt
    V = V + dV_dt*dt
    
    return State(R=r, L=l, E=e, V=v)